# GT Educação

O GT Educação enviou uma série de formulários com indicadores desejados para o painel. A maioria desses indicadores estão disponíveis no Portal de Dados Abertos da Prefeitura de São Paulo (http://dados.prefeitura.sp.gov.br/).

Portanto, vamos carregar o módulo de downloads do portal de dados abertos.

In [1]:
from core.downloads import dadosabertos as da

# Número de alunos da Rede Municipal de Ensino

O primeiro indicador é o número de alunos da rede municipal, segmentado por nível educacional (Creche, Pré Escola, Ensino Fundamental I, Ensino Fundamental II, EJA I, EJA II, Ensino Médio, Ed. Prof.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/demanda-e-matriculas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.